<a href="https://colab.research.google.com/github/Choiyh1116/AI_capstone/blob/CHOI/Code/VideoCheck3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pytube
from IPython.display import YouTubeVideo
from google.colab import drive
import os, sys
from pytube import YouTube
drive.mount('/content/drive')
path = "/content/drive/My Drive/video"
title = "4"
attitle = title + ".mp4"
os.chdir(path)

youtube = YouTube("https://www.youtube.com/watch?v=-NfZt6pUexo&ab_channel=JaehoKim")

print(youtube.title)
video = youtube.streams.get_highest_resolution()
vd = video.download()
os.rename(vd, attitle)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1-1 공학설계의배경(창의적기초공학설계)


In [4]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pylab as plt
import time

vidName = path + "/" + title + ".mp4"
methods = ['CORREL', 'CHISQR', 'INTERSECT', 'BHATTACHARYYA', 'EMD']
hists = []

print(vidName)

#csv파일관련
cor, chi, ist, bha, emd, sct = list(), list(), list(), list(), list(), list()
csvName = path + "/" + title + ".csv"
filecsv = open(csvName, "w")

print(csvName)

#영상 못불러오면 재시도
"""if not vidcap.isOpened():
    print("Could not Open..")
    exit(0)"""

vidcap = cv2.VideoCapture(vidName)
while(not vidcap.isOpened()):
  print("Could not Open..")
  time.sleep(1)
  vidcap = cv2.VideoCapture(vidName)

count = 0



#각각 초당프레임, 영상 총 프레임 수, 계산된 영상 길이(초단위)
while(True):
  fps = int(vidcap.get(cv2.CAP_PROP_FPS))
  if(fps!=0):
    break
  else:
    print("file is not loading...")
totalf = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
totalt = totalf/fps
#print("FPS : %d\t영상길이 : %f"%(fps,totalt))

#코드돌리는시간이 너무 길다면, 조절한다 -> frameGap
frameGap = fps*3
gapCount = int(frameGap-1)

while(vidcap.isOpened()):
    if(gapCount==0):
        ret, image = vidcap.read()
        gapCount = int(frameGap-1)
    else:
        ret = vidcap.grab()
        gapCount -= 1
        continue
    if ret:
        if(int (vidcap.get(1) % frameGap == 0)):
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
            cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
            hists.append(hist)
                
            if(len(hists)==2):
                query = hists[0]
                for index, name in enumerate(methods):
    
                    for i, histogram in enumerate(hists):
                        if(i == 0):
                            continue
                        compare = cv2.compareHist(query, histogram, index) 
        
                        if(index == cv2.HISTCMP_INTERSECT):
                            compare = compare/np.sum(query)
                        
                        if(name == 'CORREL'):
                            cor.append(compare)
                        if(name == 'CHISQR'):
                            chi.append(compare)
                        if(name == 'INTERSECT'):
                            ist.append(compare)
                        if(name == 'BHATTACHARYYA'):
                            bha.append(compare)
                        if(name == 'EMD'):
                            emd.append(compare)
                            sct.append(0)
                            
                del(hists[0])
            
            
            count += 1
    else:
        break
        
filecsv.write('CORREL,CHISQR,INTERSECT,BHATTACHARYYA,EMD,Screen_Transition\n')
filecsv.write(str(fps)+','+str(totalf)+','+str(frameGap)+',0,0,0\n')
for i in range(len(cor)):
    filecsv.write(str(cor[i]) + ',' + str(chi[i]) + ',' + str(ist[i]) + ',' + str(bha[i]) + ',' + str(emd[i]) + ',' + str(sct[i]) + '\n')
    
filecsv.close()

del(cor,chi,ist,bha,emd,sct)

vidcap.release()

/content/drive/My Drive/video/4.mp4
/content/drive/My Drive/video/4.csv


In [5]:
import pickle
import joblib
import pandas as pd

df = pd.read_csv(csvName)
dl = df.loc[0].to_list()
df.drop(0,axis=0,inplace=True)

fps, totalf, frameGap = dl[0], dl[1], dl[2]

r1 = ['CORREL','CHISQR','INTERSECT','BHATTACHARYYA','EMD']
#r2 = ['Screen_Transition']

x = df[r1]
#y = df['Screen_Transition']

model = joblib.load('/content/drive/MyDrive/model/model2.pkl')

pred = model.predict(x)

predDf = pd.DataFrame(pred)
predDf.columns = ['Screen_Transition']

predList = predDf.loc[predDf['Screen_Transition']==1].index.tolist()

acttime = []
for i in range(len(predList)):
    sec = predList[i]*frameGap/fps
    minute = int(sec // 60)
    sec = sec - (minute*60)
    acttime.append(str(minute)+'분 '+str(sec)+'초')

print("총 장면전환 개수 : %d", len(predList))

for i in range(len(acttime)):
    print('\t\t'+acttime[i])

x = pd.concat([x,predDf],axis=1)

x.to_csv(csvName, mode='w')

FileNotFoundError: ignored